In [ ]:
import csv
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git
import snscrape.modules.twitter as sntwitter
import itertools
import pandas as pd
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-uh_mxh_f
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-uh_mxh_f
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 3dd9c28e31b8babeb2a187fbae994d9717ded168
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for snscrape: filename=snscrape-0.6.2.20230321.dev3+g3dd9c28-py3-none-any.whl size=72466 sha256=9a905720f7ad2b4699b837c3183b2982bfbb7777a681cd337ae5e2ca3bbe1b44
  Stored in directory: /tmp/pip-ephem-wheel-cache-z975d7aq/wheels/1a/ba/e2/39fa3a11802c4a622f2efc8be3f5ff854481051d0b4c95c1fd
Successfully built snscrape


In [1]:
# mount the drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Mounted at /gdrive


In [ ]:
# change drive to my working dirtectory
%cd /gdrive/MyDrive/DSKU/datasets

/gdrive/MyDrive/DSKU/datasets


In [ ]:
OUTPUT_FILE = "/gdrive/MyDrive/DSKU/datasets/results.csv"

In [ ]:
def save_tweets(query, n_tweets):
    """
    Finds the latest tweets according to a query and saves them in a csv file
    :param query: the query to look for
    :param n_tweets: number of tweets to scrape
    """
    with open(OUTPUT_FILE, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'date', 'username', 'content', 'url'])
        
        tweets_gen = sntwitter.TwitterSearchScraper(query).get_items()
        top_tweets = itertools.islice(tweets_gen, n_tweets)
        
        for tweet in top_tweets:
            writer.writerow([tweet.id, tweet.date, tweet.username, tweet.content, tweet.url])
        
        print('Done!')

In [ ]:
query = "#foreignaid"

In [ ]:
save_tweets(query=query, n_tweets=10)

Done!


<ipython-input-5-85c67548a1b9>:15: DeprecatedFeatureWarning: username is deprecated, use user.username instead
  writer.writerow([tweet.id, tweet.date, tweet.username, tweet.content, tweet.url])
<ipython-input-5-85c67548a1b9>:15: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  writer.writerow([tweet.id, tweet.date, tweet.username, tweet.content, tweet.url])


In [ ]:
tweets_gen = sntwitter.TwitterSearchScraper(query).get_items()

In [ ]:
#type(tweet)

In [ ]:
#[tweet for tweet in tweets_gen]

In [ ]:
#tweet

**Advanced**

Advanced Search reference: https://github.com/igorbrigadir/twitter-advanced-search

In [ ]:
def advancedTweetSearch(query, loc, start, end, lang="en"):
    TWEET_STOR = []
    LOC_STOR = []
    
    scraped_tweets = sntwitter.TwitterSearchScraper(f'{query} -is:retweet geocode:"{loc}"since:{start} until:{end} lang:"{lang}"').get_items()
    
    sliced_scraped_tweets = itertools.islice(scraped_tweets, 2000000)
    
    
    for tweet in sliced_scraped_tweets:
        try:
            LOC_STOR.append(json.loads(tweet.json())['place']['country'])
            TWEET_STOR.append(json.loads(tweet.json()))
        except TypeError:
            continue
        
    return TWEET_STOR, LOC_STOR

In [ ]:
query = "#usaid OR #usforeignaid OR #foreignaid OR #foreignassistance OR #ushumanitarianaid OR #humanitarianaid OR #usmilitaryaid OR #militaryaid OR usaid OR usforeignaid OR foreignaid OR foreignassistance OR ushumanitarianaid OR humanitarianaid OR usmilitaryaid OR militaryaid OR aideffectiveness OR #aideffectiveness" #OR #economy"
start = '2017-01-01'
end = '2023-01-01'
loc = "2.723583, 113.159180, 3500km"

In [ ]:
%time tweet_stor, location_details = advancedTweetSearch(query=query, start=start, end=end, loc=loc)

CPU times: user 9.58 s, sys: 181 ms, total: 9.76 s
Wall time: 4min 28s


In [ ]:
pd.Series(location_details).value_counts()

India                             1812
Pakistan                           487
Bangladesh                         204
Nepal                              133
Sri Lanka                           62
Republic of the Philippines         39
Maldives                            23
Australia                           21
Taiwan                              15
People's Republic of China          12
                                    11
Indonesia                            3
Papua New Guinea                     2
Federated States of Micronesia       2
Japan                                1
Bhutan                               1
Malaysia                             1
dtype: int64

In [ ]:
tweet_stor[0]

{'_type': 'snscrape.modules.twitter.Tweet',
 'url': 'https://twitter.com/SyedNasserWaja1/status/1609089408799588352',
 'date': '2022-12-31T07:29:45+00:00',
 'rawContent': 'MASHALLAH successful Trip Of Honorable Farooq Mughal State Representative of Georgia I am honored to Received Shield of Honor State of Georgia.\n#syednasserwajahat #ampakbdf @usconsulatekhi @PakinUSA @ForeignOfficePk @Masood__Khan @USAID @tdap_official @usembislamabad https://t.co/CjUbBAiEGI',
 'renderedContent': 'MASHALLAH successful Trip Of Honorable Farooq Mughal State Representative of Georgia I am honored to Received Shield of Honor State of Georgia.\n#syednasserwajahat #ampakbdf @usconsulatekhi @PakinUSA @ForeignOfficePk @Masood__Khan @USAID @tdap_official @usembislamabad https://t.co/CjUbBAiEGI',
 'id': 1609089408799588352,
 'user': {'_type': 'snscrape.modules.twitter.User',
  'username': 'SyedNasserWaja1',
  'id': 1003724679843020800,
  'displayname': 'Syed Nasser Wajahat',
  'rawDescription': 'Co. Founder Gl

In [ ]:
len(tweet_stor)

2829

In [ ]:
# write the fields of interest to the file
header= ['url', 'username' 'date', 'rawcontent', 'viewCount', 'country', 'countrycode']
with open(OUTPUT_FILE, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for tweet in range(len(tweet_stor)):
          #print(tweet)
            writer.writerow([tweet_stor[tweet]['url'], tweet_stor[0]['user']['username'],tweet_stor[tweet]['date'], tweet_stor[tweet]['rawContent'], tweet_stor[tweet]['viewCount'], tweet_stor[tweet]['place']['country'], tweet_stor[tweet]['place']['countryCode']])

In [ ]:
# brazil

#loc = "-8.581021,-51.328125, 4500km"
#loc = "-8.581021,-51.328125, 4500km"
loc="-23.569995, -69.114700, 4500km"

In [ ]:
%time tweet_stor_b, location_details_b = advancedTweetSearch(query=query, start=start, end=end, loc=loc)

CPU times: user 51.8 ms, sys: 5.02 ms, total: 56.9 ms
Wall time: 4.11 s


In [ ]:
pd.Series(location_details_b).value_counts()

<ipython-input-70-d2559bd5c043>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(location_details_b).value_counts()


Series([], dtype: int64)